GLIDE
====

**GLIDE: Towards Photorealistic Image Generation and Editing with Text-Guided Diffusion Models**

 * Paper: https://arxiv.org/abs/2112.10741
 * Code: https://github.com/openai/glide-text2im

![GLIDE Examples](../assets/glide_examples.jpg)


```bash
conda create -n glide python=3.11 -y
conda activate glide
pip install git+https://github.com/openai/glide-text2im
pip install pyyaml
```